In [1]:
import requests

From the command line, start the Flask application using development server.

    python3 qdotweb.py
    
The default address is http://localhost:5000 . That page will show your computer name, the local time, and a list of available ports. We can also get a list of available ports and any \*IDN? resopnses from \instruments

In [2]:
r = requests.get('http://localhost:5000/instruments')
jr = r.json()
print(jr['address'], jr['IDN'])

['ASRL1::INSTR', 'ASRL2::INSTR', 'GPIB0::5::INSTR', 'url'] ['', '', 'LSCI,MODEL370,370014,04102008\n', '']


There we see that at 'GPIB0::5::INSTR' (GPIB board 0, address 5) there is a LakeShore 370 Resistance Bridge. Let's talk to it. To do that we will use the HTTP POST method to send some keyword arguments to the page.

In [3]:
# connect the instrument

r = requests.post('http://localhost:5000/connect', data={'address':'GPIB0::5::INSTR'})
jr = r.json()
print(jr)

{'data': 'CONNECTED', 'errors': '', 'address': 'GPIB0::5::INSTR', 'command': 'connect'}


In [8]:
# request the resistance reading from channel 2

r = requests.post('http://localhost:5000/query', data={'address':'GPIB0::5::INSTR', 'command':'RDGR? 2'})
jr = r.json()
print(jr)

{'data': '+4.99240E+03\n', 'errors': '', 'address': 'GPIB0::5::INSTR', 'command': 'RDGR? 2'}


In [9]:
# close the instrument

r = requests.post('http://localhost:5000/close', data={'address':'GPIB0::5::INSTR'})
jr = r.json()
print(jr)

{'data': 'CLOSED', 'errors': '', 'address': 'GPIB0::5::INSTR', 'command': 'close'}


Obviously, there is no need to print all of that data every time. A real program might look something like the following.

In [10]:
import time

r = requests.post('http://localhost:5000/connect', data={'address':'GPIB0::5::INSTR'})

tstart = time.time()
N=100

for i in range(N):
    r = requests.post('http://localhost:5000/query', data={'address':'GPIB0::5::INSTR', 'command':'RDGK? 2'})
    data = r.json()['data']
print('{} s per request'.format((time.time()-tstart)/N))

r = requests.post('http://localhost:5000/close', data={'address':'GPIB0::5::INSTR'})

0.019798378944396972 s per request
